In [1]:
import numpy as np
import pythreejs as p3
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import FloatText, BoundedFloatText, Button, VBox, FloatSlider, Dropdown, Label, HBox, ToggleButton
import numpy as np
from scipy.integrate import cumtrapz
import numpy as np
from scipy.integrate import cumtrapz

x = np.linspace(-2 * np.pi, 2 * np.pi, 200)
amp = 8.0  
freq = 5.0 
phase_degrees = 0.0
phase = np.deg2rad(phase_degrees)
y = amp * np.sin(freq * x + phase)
z = np.zeros_like(x)
x_min, x_max = min(x), max(x)
y_min, y_max = min(y), max(y)
z_min, z_max = min(z), max(z)
axis_size = max(x_max, y_max) + 1
arrow_length = axis_size / 10
scene = p3.Scene()
background_color = 'black'
scene.background = background_color
camera = p3.PerspectiveCamera(position=[axis_size * 1.5, axis_size * 1.5, axis_size * 1.5], aspect=1)
scene.add(camera)
sine_wave_geometry = p3.BufferGeometry(attributes=dict(
    position=p3.BufferAttribute(np.array([x, y, z]).T.astype('float32'))))
sine_wave_material = p3.LineBasicMaterial(color='white')
sine_wave_line = p3.Line(geometry=sine_wave_geometry, material=sine_wave_material, linewidth=500)
axes = p3.AxesHelper(size=axis_size)
axes.showLabels = True
neg_axes = p3.AxesHelper(size=-axis_size)
neg_axes.showLabels = True
domain_toggle_button = ToggleButton(value=False, description='Time Domain', button_style='info', layout=widgets.Layout(width='auto'))
arrow_x = p3.ArrowHelper(dir=[1, 0, 0], origin=[axis_size, 0, 0], length=arrow_length, color='red')
arrow_y = p3.ArrowHelper(dir=[0, 1, 0], origin=[0, axis_size, 0], length=arrow_length, color='green')
arrow_z = p3.ArrowHelper(dir=[0, 0, 1], origin=[0, 0, axis_size], length=arrow_length, color='blue')
neg_arrow_x = p3.ArrowHelper(dir=[-1, 0, 0], origin=[-axis_size, 0, 0], length=arrow_length, color='red')
neg_arrow_y = p3.ArrowHelper(dir=[0, -1, 0], origin=[0, -axis_size, 0], length=arrow_length, color='green')
neg_arrow_z = p3.ArrowHelper(dir=[0, 0, -1], origin=[0, 0, -axis_size], length=arrow_length, color='blue')
neg_x_points = np.array([[-axis_size, 0, 0], [0, 0, 0]])
neg_x_geometry = p3.BufferGeometry(attributes=dict(position=p3.BufferAttribute(neg_x_points.astype('float32'))))
neg_x_material = p3.LineBasicMaterial(color='red', linewidth=3)
neg_x_line = p3.Line(geometry=neg_x_geometry, material=neg_x_material)
grid_lines = p3.WireframeGeometry(
    positions=np.array([[-axis_size, 0, 0], [axis_size, 0, 0], [0, -axis_size, 0], [0, axis_size, 0],
                        [0, 0, -axis_size], [0, 0, axis_size]]).astype('float32'),
    lines=np.array([[0, 1], [2, 3], [4, 5]]).astype('uint32')
)
grid_material = p3.LineBasicMaterial(color='gray')
grid_line = p3.Line(geometry=grid_lines, material=grid_material)
scene.add(sine_wave_line)
scene.add(axes)
scene.add(arrow_x)
scene.add(arrow_y)
scene.add(arrow_z)
scene.add(neg_axes)
scene.add(neg_arrow_x)
scene.add(neg_arrow_y)
scene.add(neg_arrow_z)
scene.add(neg_x_line)
scene.add(grid_line)
renderer = p3.Renderer(camera=camera, scene=scene, controls=[p3.OrbitControls(controlling=camera)])
renderer.width = 800
renderer.height = 600
def update_camera_position(button_click):
    if button_click['new']:
        domain_toggle_button.description = 'Frequency Domain'
        camera.position = camera_position_freq
    else:
        domain_toggle_button.description = 'Time Domain'
        camera.position = camera_position_time

domain_toggle_button.observe(update_camera_position, names='value')

camera_position_time = [axis_size * 0, axis_size * 1, axis_size * 5]
camera_position_freq = [axis_size * 5, axis_size * 1, axis_size * 0]
update_camera_position({'new': domain_toggle_button.value})
amp_input = FloatText(value=amp, description='Amplitude:')
freq_input = FloatText(value=freq, description='Frequency:')
phase_input = FloatText(value=phase_degrees, description='Phase (degrees):')
base_input = FloatText(value=1.0, description='Base:')
exponent_input = FloatText(value=1.0, description='Exponent:')
base_input.layout.visibility = 'hidden'
exponent_input.layout.visibility = 'hidden'
function_dropdown = widgets.Dropdown(
    options=['Sine Wave', 'Cosine', 'Square', 'Signum', 'Exponential', 'Impulse', 'Sinc', 'Rectangular', 'Triangular'],
    description='Function:'
)

continuous_discrete_toggle = ToggleButton(value=False, description='Continuous Signal', button_style='info', layout=widgets.Layout(width='auto'))

def get_signal_sample(x_val, signal_type):
    if signal_type == 'Sine Wave':
        return amp * np.sin(freq_input.value * x_val + phase)
    elif signal_type == 'Cosine':
        return amp * np.cos(freq * x_val + phase)
    elif signal_type == 'Square':
        return amp * np.sign(np.sin(freq * x_val + phase))
    elif signal_type == 'Signum':
        return amp * np.sign(x_val)
    elif signal_type == 'Exponential':
        return amp * np.exp(x_val)
    elif signal_type == 'Impulse':
        return amp if x_val == 0 else 0
    elif signal_type == 'Sinc':
        return amp * np.sinc(x_val)
    elif signal_type == 'Rectangular':
        period = 2 * np.pi / freq
        return amp * np.where((x_val % period) < period / 2, 1, -1)
    elif signal_type == 'Triangular':
        period = 2 * np.pi / freq
        return amp * 2.0 / period * (np.abs(x_val % period - period / 2.0) - period / 4.0)
    else:
        return 0 

def update_signal_representation(button_click):
    selected_function = function_dropdown.value
    positive_time = domain_toggle_button.value

    if button_click['new']:
        continuous_discrete_toggle.description = 'Discrete Signal'

        x_discrete = np.arange(-11, 10)  
        remove_existing_lines()
        lines = []
        for x_val in x_discrete:
            y_val = get_signal_sample(x_val, selected_function)
            line_geometry = p3.Geometry(vertices=[[x_val, 0, 0], [x_val, y_val, 0]])
            line_material = p3.LineBasicMaterial(color='white')
            line = p3.Line(geometry=line_geometry, material=line_material)
            lines.append(line)
            scene.add(line)
            
        sine_wave_geometry.attributes['position'].array = np.array([[], [], []]).T.astype('float32')
    else:  
        continuous_discrete_toggle.description = 'Continuous Signal'
        remove_existing_lines()
        y_continuous = update_sine_wave(selected_function)
        sine_wave_geometry.attributes['position'].array = np.array([x, y_continuous, z]).T.astype('float32')

def remove_existing_lines():
    for child in scene.children:
        if isinstance(child, p3.Line):
            scene.remove(child)

def update_input_from_bounds(change):
    amp_input.value = amp_bounds.value
    freq_input.value = freq_bounds.value
    phase_input.value = phase_bounds.value

amp_bounds = BoundedFloatText(value=amp, min=-1e6, max=1e6, description='Bounds:')
freq_bounds = BoundedFloatText(value=freq, min=0.1, max=1e6, description='Bounds:')
phase_bounds = BoundedFloatText(value=phase_degrees, min=-360.0, max=360.0, description='Bounds:')
amp_bounds.observe(update_input_from_bounds, names='value')
freq_bounds.observe(update_input_from_bounds, names='value')
phase_bounds.observe(update_input_from_bounds, names='value')
update_button = Button(description="Update Domains")

def update_domains(button_click):
    positive_time = not button_click['new']
    current_freq = max(freq_input.value, 0.1)
    x_shifted = x + positive_time
    y = amp_input.value * np.sin(current_freq * x_shifted + np.deg2rad(phase_input.value))
    sine_wave_geometry.attributes['position'].array = np.array([x_shifted, y, z]).T.astype('float32')

domain_toggle_button.observe(update_domains, names='value')


def update_sine_wave(change):
    selected_function = function_dropdown.value
    positive_time = domain_toggle_button.value
    current_amp = amp_input.value
    current_freq = max(freq_input.value, 0.1)
    current_phase_degrees = phase_input.value
    current_phase = np.deg2rad(current_phase_degrees)
    time_shift = 0.0

    if selected_function == 'Exponential':
        base_input.layout.visibility = 'visible'
        exponent_input.layout.visibility = 'visible'
        amp_input.layout.visibility = 'hidden'
        freq_input.layout.visibility = 'hidden'
        phase_input.layout.visibility = 'hidden'
    elif selected_function == 'Signum':
        amp_input.layout.visibility = 'visible'
        freq_input.layout.visibility = 'hidden'
        phase_input.layout.visibility = 'hidden'
        base_input.layout.visibility = 'hidden'
        exponent_input.layout.visibility = 'hidden'
    else:
        amp_input.layout.visibility = 'visible'
        freq_input.layout.visibility = 'visible'
        phase_input.layout.visibility = 'visible'
        base_input.layout.visibility = 'hidden'
        exponent_input.layout.visibility = 'hidden'

    if selected_function == 'Sine Wave' and continuous_discrete_toggle.value==False:
        y = current_amp * np.sin(current_freq * (x + time_shift) + np.deg2rad(current_phase_degrees))
    elif selected_function == 'Cosine':
        y = current_amp * np.cos(current_freq * (x + time_shift) + np.deg2rad(current_phase_degrees))
    elif selected_function == 'Square':
        y = current_amp * np.sign(np.sin(current_freq * (x + time_shift) + np.deg2rad(current_phase_degrees)))
    elif selected_function == 'Signum':
        y = current_amp * np.sign(x + time_shift)
    elif selected_function == 'Exponential':
        amp = amp_input.value
        base = base_input.value
        exponent = exponent_input.value
        y=np.exp(x) * (x > 0) - np.exp(-x) * (x < 0)
        sine_wave_geometry.attributes['position'].array = np.array([x, y, z]).T.astype('float32')  # Adjust the time shift if necessary
    elif selected_function == 'Impulse':
        y = current_amp * np.zeros_like(x)
        y[len(x) // 2] = current_amp
    elif selected_function == 'Sinc':
        y = current_amp * np.sinc(x + time_shift)
    elif selected_function == 'Rectangular':
        period = 2 * np.pi / current_freq
        y = current_amp * np.where(((x + time_shift) % period) < period / 2, 1, -1)
    elif selected_function == 'Triangular':
        period = 2 * np.pi / current_freq
        y = current_amp * 2.0 / period * (
                np.abs((x + time_shift) % period - period / 2.0) - period / 4.0)
    else:
        y = np.zeros_like(x)

    y -= np.mean(y)
    sine_wave_geometry.attributes['position'].array = np.array([x, y, z]).T.astype('float32')
    axis_size = max(x_max, y_max, -y_min) + 1
    arrow_x.length = arrow_y.length = arrow_z.length = neg_arrow_x.length = neg_arrow_y.length = neg_arrow_z.length = axis_size / 10
    
amp_input.observe(update_sine_wave, names='value')
freq_input.observe(update_sine_wave, names='value')
phase_input.observe(update_sine_wave, names='value')
function_dropdown.observe(update_sine_wave, names='value')

amp_widgets = VBox([amp_input, freq_input, phase_input, domain_toggle_button])
exponential_widgets = VBox([base_input, exponent_input])
input_widgets = VBox([
    function_dropdown, amp_widgets, exponential_widgets])

carrier_amp_input = FloatText(value=1.0, description='Amplitude:')
carrier_freq_input = FloatText(value=10.0, description='Frequency:')
carrier_phase_input = FloatText(value=0.0, description='Phase (degrees):')
carrier_modiation_index= FloatText(value=0.5, description='Modulation Index:',max=1.0,min=0.0,step=0.1)
carrier_wave_geometry = p3.BufferGeometry(attributes=dict(
    position=p3.BufferAttribute(np.array([x, np.zeros_like(x), z]).T.astype('float32'))
))
carrier_wave_material = p3.LineBasicMaterial(color='orange')  
carrier_wave_line = p3.Line(geometry=carrier_wave_geometry, material=carrier_wave_material, linewidth=500)
scene.add(carrier_wave_line)

def update_carrier_wave(change):
    carrier_amp = carrier_amp_input.value
    carrier_freq = max(carrier_freq_input.value, 0.1)
    carrier_phase_degrees = carrier_phase_input.value
    carrier_phase = np.deg2rad(carrier_phase_degrees)

    carrier_wave = carrier_amp * np.sin(carrier_freq * x + carrier_phase)
    carrier_wave -= np.mean(carrier_wave)
    carrier_wave_geometry.attributes['position'].array = np.array([x, carrier_wave, z]).T.astype('float32')
        
carrier_amp_input.observe(update_carrier_wave, names='value')
carrier_freq_input.observe(update_carrier_wave, names='value')
carrier_phase_input.observe(update_carrier_wave, names='value')

def calculate_nyquist_rate(modulated_freq):
    nyquist_rate = 2.0 * modulated_freq+30
    return nyquist_rate

input_visibility_switch = ToggleButton(value=True, description='Input Visibility', button_style='info', layout=widgets.Layout(width='auto'))
carrier_visibility_switch = ToggleButton(value=True, description='Carrier Visibility', button_style='info', layout=widgets.Layout(width='auto'))
modulated_visibility_switch = ToggleButton(value=True, description='Modulated Visibility', button_style='info', layout=widgets.Layout(width='auto'))

def update_visibility(change):
    sine_wave_material.visible = True if input_visibility_switch.value else False 
    carrier_wave_material.visible = True if carrier_visibility_switch.value else False
    modulated_signal_material.visible = True if modulated_visibility_switch.value else False
    renderer.render(scene, camera)

input_visibility_switch.observe(update_visibility, names='value')
carrier_visibility_switch.observe(update_visibility, names='value')
modulated_visibility_switch.observe(update_visibility, names='value')
modulation_dropdown = Dropdown(options=['None', 'AM', 'FM', 'PM', 'ASK', 'FSK', 'PSK', 'QPSK', 'QAM'], value='None', description='Modulation:')
modulated_signal_material = p3.LineBasicMaterial(color='blue', linewidth=5)
modulated_signal_geometry = p3.BufferGeometry(attributes=dict(
    position=p3.BufferAttribute(np.array([x, np.zeros_like(x), z]).T.astype('float32'))
))

modulated_signal_line = p3.Line(geometry=modulated_signal_geometry, material=modulated_signal_material, linewidth=5)
scene.add(modulated_signal_line)

def update_modulated_frequency(change):
    modulated_freq = freq_input.value  # You may need to adjust this based on your specific case
    nyquist_rate = calculate_nyquist_rate(modulated_freq)
    x = np.linspace(-2 * np.pi, 2 * np.pi, int(1000 * (2 * np.pi / nyquist_rate)))
    sine_wave_geometry.attributes['position'].array = np.array([x, y, z]).T.astype('float32')

def update_modulated_signal(change):
    selected_modulation = modulation_dropdown.value
    modulated_amp = amp_input.value
    modulated_freq = calculate_nyquist_rate(freq_input.value)
    modulated_phase_degrees = phase_input.value
    modulated_phase = np.deg2rad(modulated_phase_degrees)
    selected_modulation = modulation_dropdown.value
    carrier_wave = carrier_amp_input.value * np.sin(carrier_freq_input.value * x + np.deg2rad(carrier_phase_input.value))
    Message_wave = amp_input.value * np.sin(freq_input.value * x + np.deg2rad(phase_input.value))

    if selected_modulation == 'AM':
        modulated_signal = (1 + carrier_modiation_index.value * Message_wave) * carrier_wave
    elif selected_modulation == 'FM':
        freq_deviation = carrier_modiation_index.value * freq_input.value
        phase_signal = cumtrapz(Message_wave, x, initial=0)
        modulated_signal = modulated_amp * np.sin(2 * np.pi * carrier_freq_input.value * x + freq_deviation * phase_signal)
    elif selected_modulation == 'PM':
        modulated_signal = modulated_amp * np.sin(
            2 * np.pi * carrier_freq_input.value * x + modulated_phase + carrier_modiation_index.value * Message_wave)
    elif selected_modulation == 'ASK':
        modulated_signal = (1 + carrier_modiation_index.value * Message_wave) * carrier_amp_input.value * np.sin(
            2 * np.pi * carrier_freq_input.value * x + np.deg2rad(carrier_phase_input.value))
    elif selected_modulation == 'FSK':
        modulated_signal = modulated_amp * np.sin(
            2 * np.pi * modulated_freq * x + modulated_phase + carrier_modiation_index.value * Message_wave)
    elif selected_modulation == 'PSK':
        modulated_signal = modulated_amp * np.sin(
            2 * np.pi * modulated_freq * x + modulated_phase + np.pi * carrier_modiation_index.value * Message_wave)
    elif selected_modulation == 'QPSK':
        in_phase = amp_input.value * np.sin(2 * np.pi * freq_input.value * x + np.deg2rad(phase_input.value))
        quadrature = amp_input.value * np.cos(2 * np.pi * freq_input.value * x + np.deg2rad(phase_input.value))
        modulated_signal = np.sqrt(in_phase ** 2 + quadrature ** 2) * modulated_amp * np.sin(
            2 * np.pi * modulated_freq * x + modulated_phase)
    elif selected_modulation == 'QAM':
        in_phase = amp_input.value * np.sin(2 * np.pi * freq_input.value * x + np.deg2rad(phase_input.value))
        quadrature = amp_input.value * np.cos(2 * np.pi * freq_input.value * x + np.deg2rad(phase_input.value))
        modulated_signal = (in_phase + 1j * quadrature) * modulated_amp * np.sin(
            2 * np.pi * modulated_freq * x + modulated_phase)
    else:
        modulated_signal = np.zeros_like(x)

    modulated_signal_geometry.attributes['position'].array = np.array([x, modulated_signal, z]).T.astype('float32')

    renderer.render(scene, camera)

modulation_dropdown.observe(update_modulated_signal, names='value')


def update_modulation_options(change):
    selected_function = function_dropdown.value
    modulation_options_square = ['ASK', 'PSK', 'QAM', 'QPSK', 'FSK']
    modulation_options_other = ['AM', 'FM', 'PM']

    if selected_function == 'Square':
        modulation_dropdown.options = modulation_options_square
    else:
        modulation_dropdown.options = modulation_options_other

function_dropdown.observe(update_modulation_options, names='value')
update_modulation_options({'new': function_dropdown.value})


input_label = Label(value="Input for information signal")
carrier_label = Label(value="Carrier Wave")
signal_widgets = VBox([input_label, function_dropdown, amp_widgets, exponential_widgets, modulation_dropdown, continuous_discrete_toggle ])
carrier_widgets = VBox([carrier_label, carrier_amp_input, carrier_freq_input, carrier_phase_input,carrier_modiation_index])
input_widgets = HBox([signal_widgets, carrier_widgets])
continuous_discrete_toggle.observe(update_signal_representation, names='value')
output_renderer = renderer
final_layout = VBox([input_widgets, output_renderer])
final_layout.children += (input_visibility_switch, carrier_visibility_switch, modulated_visibility_switch)
display(final_layout)
